In [15]:
%conda install pandas gdal scikit-learn scikit-image scipy matplotlib seaborn seaborn-image

Channels:
 - defaults
Platform: linux-64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - seaborn-image

Current channels:

  - https://repo.anaconda.com/pkgs/main
  - https://repo.anaconda.com/pkgs/r

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from pathlib import Path
from dotenv import load_dotenv
import rasterio
import numpy as np
from rasterio.plot import show
import os
import matplotlib.pyplot as plt
import pandas as pd

np.seterr(divide='ignore', invalid='ignore')
os.chdir("/home/me/workspace/det_remota/trabalho_final")

In [2]:
infrared_images = [
    rasterio.open(image_file)
    for image_file in Path(os.getcwd()).glob('data/sentinel2/COLOR_INFRARED_*.tiff')
]
swir_images = [
    rasterio.open(image_file)
    for image_file in Path(os.getcwd()).glob('data/sentinel2/SWIR_*.tiff')
]

In [21]:
TIME_INDEXES = [img.files[0].split('/')[-1].replace('SWIR_', '') for img in swir_images]
ALL_BANDS_OUTPUT_PATH = Path('data/sentinel2/pre_processed/all_bands')

OUT_CHANNEL_NAMES = [
    'band_3',
    'band_4',
    'band_8',
    'band_11',
    'band_12',
    'ndwi',
    'mndwi',
    'ndvi',
    'ndmi',
    'msavi2',
]

def find_image(image_list: list, filename:str):
    for img in image_list:
        if filename in img.files[0].split('/')[-1]:
            return img
    return None

def apply_constrast(channel_data):
    min_max = (channel_data - np.min(channel_data)) / (np.max(channel_data) - np.min(channel_data))
    return np.round(255 * min_max, 0)

def msavi2(channel_nir, channel_red):
    """
    MSAVI2 = (1/2)*(2(NIR+1)-sqrt((2*NIR+1)^2-8(NIR-Red)))
    """
    return (1/2)*(2*(channel_nir+1) - np.sqrt(np.power(2*channel_nir+1, 2) - 8*(channel_nir-channel_red)))

def extract_channels(time_index:str):
    """
    COLOR_INFRARED is RGB 8-4-3
    SWIR is RGB 12-11-4
    """
    infrared_img = find_image(infrared_images, time_index)
    swir_img = find_image(swir_images, time_index)

    band_8, band_4, band_3 = (infrared_img.read(x) for x in range(1,4))
    band_12, band_11 = (swir_img.read(x) for x in range(1,3))
    
    band_3 = apply_constrast(band_3)
    band_4 = apply_constrast(band_4)
    band_8 = apply_constrast(band_8)
    band_11 = apply_constrast(band_11)
    band_12 = apply_constrast(band_12)
    
    channels_values = [
        band_3,
        band_4,
        band_8,
        band_11,
        band_12,
        apply_constrast((band_3 - band_8) / (band_3 + band_8)),
        apply_constrast((band_3 - band_11) / (band_3 + band_11)),
        apply_constrast((band_8 - band_4) / (band_8 + band_4)),
        apply_constrast((band_8 - band_11) / (band_8 + band_11)),
        apply_constrast(msavi2(band_4, band_3)),
    ]

    out_filename = ALL_BANDS_OUTPUT_PATH.joinpath(time_index)
    profile = infrared_img.profile.copy()
    profile.update(count=len(channels_values))

    with rasterio.open(out_filename, "w", **profile) as out_tiff:
        for idx, values in enumerate(channels_values):
            out_tiff.write_band(idx+1, values)


pd.DataFrame({
    'Band': range(1, len(OUT_CHANNEL_NAMES)+1),
    'Description': OUT_CHANNEL_NAMES
}).to_csv(ALL_BANDS_OUTPUT_PATH.joinpath('bands.csv'), index=False)


for time_index in TIME_INDEXES:
    extract_channels(time_index)